In [83]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris-dataset-dendrite/iris.csv
/kaggle/input/jason-parsing/algoparams_from_ui.json.rtf


In [84]:
import json
import csv
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor, ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, SGDClassifier, SGDRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from xgboost import XGBClassifier, XGBRegressor
import re
import string


In [85]:
!pip install pypandoc

In [86]:
import pypandoc

def convert_rtf_to_text(file_path):
    output = pypandoc.convert_file(json_file_path, 'plain', format='rtf')
    return output


json_file_path = '/kaggle/input/jason-parsing/algoparams_from_ui.json.rtf'
# json_file_path = 'algoparams_from_ui.json.rtf'
text = convert_rtf_to_text(json_file_path)


# Stripping newline and spaces to create a dictionary
stripped_text = text.replace('\n', '')
text = stripped_text.replace(' ', '')
constraint_dict = json.loads(text)

In [87]:
def feature_handling(X):
    # Removing Features based on the given input
    if(constraint_dict['design_state_data']['feature_handling']['sepal_length']['is_selected'] == False):
        print("sepal_length False")
        X = X.drop('sepal_length', axis = 1)

    if(constraint_dict['design_state_data']['feature_handling']['sepal_width']['is_selected'] == False):
        print("False")
        X = X.drop('sepal_width', axis = 1)

    if(constraint_dict['design_state_data']['feature_handling']['petal_length']['is_selected'] == False):
        print("False")
        X = X.drop('petal_length', axis = 1)

    if(constraint_dict['design_state_data']['feature_handling']['petal_width']['is_selected'] == False):
        print("False")
        X = X.drop('petal_width', axis = 1)

#     Impute the data as specified in the input values
    if(constraint_dict['design_state_data']['feature_handling']['sepal_length']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['sepal_length'] = imputer.fit_transform(X[['sepal_length']])
    elif(constraint_dict['design_state_data']['feature_handling']['sepal_length']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['sepal_length']['feature_details']['impute_value']))
        X['sepal_length'] = imputer.fit_transform(X[['sepal_length']])

        
    if(constraint_dict['design_state_data']['feature_handling']['sepal_width']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['sepal_width'] = imputer.fit_transform(X[['sepal_width']])
    elif(constraint_dict['design_state_data']['feature_handling']['sepal_width']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['sepal_width']['feature_details']['impute_value']))
        X['sepal_width'] = imputer.fit_transform(X[['sepal_width']])


    if(constraint_dict['design_state_data']['feature_handling']['petal_length']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['petal_length'] = imputer.fit_transform(X[['petal_length']])
    elif(constraint_dict['design_state_data']['feature_handling']['petal_length']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['petal_length']['feature_details']['impute_value']))
        X['petal_length'] = imputer.fit_transform(X[['petal_length']])


    if(constraint_dict['design_state_data']['feature_handling']['petal_width']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['petal_width'] = imputer.fit_transform(X[['petal_width']])
    elif(constraint_dict['design_state_data']['feature_handling']['petal_width']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['petal_width']['feature_details']['impute_value']))
        X['petal_width'] = imputer.fit_transform(X[['petal_width']])
    return X

In [88]:
# Set the algorithm parameters such that it runs as specified in the input

# Classification parameters for algorithm tuning
svc_kernels = {'linear_kernel': 'linear', 'rep_kernel': 'rbf', 'polynomial_kernel': 'poly', 'sigmoid_kernel': 'sigmoid'}

svc_regs = {'use_l1_regularization': 'l1', 'use_l2_regularization': 'l2', 'use_elastic_net_regularization': 'elasticnet'}

dc_impure = {'use_gini' : 'gini', 'use_entropy' : 'entropy'}

dc_splitter = {'use_best' : 'best', 'use_random' : 'random'}

svr_kernels = {'linear_kernel': 'linear', 'rep_kernel': 'rbf', 'polynomial_kernel': 'poly', 'sigmoid_kernel': 'sigmoid'}

clf_rf_n_est = list(range(constraint_dict['design_state_data']['algorithms']['RandomForestClassifier']['min_trees'], 
                 constraint_dict['design_state_data']['algorithms']['RandomForestClassifier']['max_trees']+5, 5))

clf_rf_max_depth = list(range(constraint_dict['design_state_data']['algorithms']['RandomForestClassifier']['min_depth'], 
                 constraint_dict['design_state_data']['algorithms']['RandomForestClassifier']['max_depth']+5,5))

clf_rf_samp_leaf = list(range(constraint_dict['design_state_data']['algorithms']['RandomForestClassifier']['min_samples_per_leaf_min_value'], 
                 constraint_dict['design_state_data']['algorithms']['RandomForestClassifier']['min_samples_per_leaf_max_value']+5,5))

clf_gb_n_est = list((constraint_dict['design_state_data']['algorithms']['GBTClassifier']['num_of_BoostingStages'][0],
                    constraint_dict['design_state_data']['algorithms']['GBTClassifier']['num_of_BoostingStages'][1]))

clf_gb_minmax_subsamp = list((constraint_dict['design_state_data']['algorithms']['GBTClassifier']['min_subsample'],
                          constraint_dict['design_state_data']['algorithms']['GBTClassifier']['max_subsample']))

clf_gb_minmax_depth = list(range(constraint_dict['design_state_data']['algorithms']['GBTClassifier']['min_depth'],
                          constraint_dict['design_state_data']['algorithms']['GBTClassifier']['max_depth']+1))

clf_lr_n_jobs = constraint_dict['design_state_data']['algorithms']['LogisticRegression']['parallelism']

clf_lr_max_iter = list(range(constraint_dict['design_state_data']['algorithms']['LogisticRegression']['min_iter'], 
                             constraint_dict['design_state_data']['algorithms']['LogisticRegression']['max_iter']+10, 10))

clf_svc_list = []
for i in svc_kernels:
    if(constraint_dict['design_state_data']['algorithms']['SVM'][i] == True):
        clf_svc_list.append(svc_kernels[i])

clf_svc_max_iter = constraint_dict['design_state_data']['algorithms']['SVM']['max_iterations']

clf_svc_l_list = []
for i in svc_regs:
    if(constraint_dict['design_state_data']['algorithms']['SGD'][i] == 'on' or
      constraint_dict['design_state_data']['algorithms']['SGD'][i] == True):
        clf_svc_l_list.append(svc_regs[i])

clf_dc_depth = list(range(constraint_dict['design_state_data']['algorithms']['DecisionTreeClassifier']['min_depth'], constraint_dict['design_state_data']['algorithms']['DecisionTreeClassifier']['max_depth']+1))

clf_dc_impure = []
for i in dc_impure:
    if(constraint_dict['design_state_data']['algorithms']['DecisionTreeClassifier'][i] == True):
        clf_dc_impure.append(dc_impure[i])

clf_dc_split = []
for i in dc_splitter:
    if(constraint_dict['design_state_data']['algorithms']['DecisionTreeClassifier'][i] == True):
        clf_dc_split.append(dc_splitter[i])
        
clf_etree_parll = constraint_dict['design_state_data']['algorithms']['extra_random_trees']['parallelism']

clf_nn_opt = constraint_dict['design_state_data']['algorithms']['neural_network']['solver']
clf_nn_alpha = constraint_dict['design_state_data']['algorithms']['neural_network']['alpha_value']
clf_nn_estop = constraint_dict['design_state_data']['algorithms']['neural_network']['early_stopping']


# Regression parameters for algorithm tuning

reg_rf_n_est = list(range(constraint_dict['design_state_data']['algorithms']['RandomForestRegressor']['min_trees'], 
                 constraint_dict['design_state_data']['algorithms']['RandomForestRegressor']['max_trees']+5, 5))

reg_rf_max_depth = list(range(constraint_dict['design_state_data']['algorithms']['RandomForestRegressor']['min_depth'], 
                 constraint_dict['design_state_data']['algorithms']['RandomForestRegressor']['max_depth']+5, 5))

reg_rf_samp_leaf = list(range(constraint_dict['design_state_data']['algorithms']['RandomForestRegressor']['min_samples_per_leaf_min_value'], 
                 constraint_dict['design_state_data']['algorithms']['RandomForestRegressor']['min_samples_per_leaf_max_value']+5,5))

reg_gb_n_est = list((constraint_dict['design_state_data']['algorithms']['GBTRegressor']['num_of_BoostingStages'][0],
                    constraint_dict['design_state_data']['algorithms']['GBTRegressor']['num_of_BoostingStages'][1]))

reg_gb_minmax_subsamp = list((constraint_dict['design_state_data']['algorithms']['GBTRegressor']['min_subsample'],
                          constraint_dict['design_state_data']['algorithms']['GBTRegressor']['max_subsample']))

reg_gb_minmax_depth = list(range(constraint_dict['design_state_data']['algorithms']['GBTRegressor']['min_depth'],
                          constraint_dict['design_state_data']['algorithms']['GBTRegressor']['max_depth']+1))

reg_lreg_njobs = constraint_dict['design_state_data']['algorithms']['LinearRegression']['parallelism']

reg_svr_list = []
for i in svr_kernels:
    if(constraint_dict['design_state_data']['algorithms']['SVM'][i] == True):
        reg_svr_list.append(svr_kernels[i])
        
reg_svr_max_iter = constraint_dict['design_state_data']['algorithms']['SVM']['max_iterations']

In [89]:
# Define a list of models with parameter grids for classification
classification_models = [
    ('RandomForestClassifier', RandomForestClassifier(), {
        'classifier__n_estimators': clf_rf_n_est,
        'classifier__max_depth': clf_rf_max_depth,
        'classifier__min_samples_leaf': clf_rf_samp_leaf
    }),
    ('GradientBoostingClassifier', GradientBoostingClassifier(), {
        'classifier__n_estimators': clf_gb_n_est,
        'classifier__learning_rate': [0.01, 0.1, 1],
        'classifier__max_depth': clf_gb_minmax_depth
    }),
    ('LogisticRegression', LogisticRegression(max_iter=1000), {
        'classifier__C': [0.1, 1, 10],
        'classifier__solver': ['liblinear', 'lbfgs'],
        'classifier__n_jobs': clf_lr_n_jobs,
        'classifier__max_iter' : clf_lr_max_iter
    }),
    ('SVC', SVC(), {
        'classifier__kernel': clf_svc_list,
        'classifier__C': [0.1, 1, 10],
        'classifier__max_iter': clf_svc_max_iter
    }),
    ('SGDClassifier', SGDClassifier(max_iter=1000, tol=1e-3), {
        'classifier__loss': ['hinge', 'log'],
        'classifier__alpha': [0.0001, 0.001, 0.01],
        'classifier__penalty' : clf_svc_l_list
    }),
    ('KNeighborsClassifier', KNeighborsClassifier(), {
        'classifier__n_neighbors': [3, 5, 7],
        'classifier__weights': ['uniform', 'distance'],
        'classifier__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
    }),
    ('DecisionTreeClassifier', DecisionTreeClassifier(), {
        'classifier__max_depth': clf_dc_depth,
        'classifier__criterion' : clf_dc_impure,
        'classifier__splitter' : clf_dc_split
    }),
    ('ExtraTreesClassifier', ExtraTreesClassifier(), {
        'classifier__n_estimators': [50, 100],
        'classifier__max_depth': [None, 10, 20],
        'classifier__n_jobs' : clf_etree_parll
    }),
    ('XGBClassifier', XGBClassifier(), {
        'classifier__n_estimators': [50, 100],
        'classifier__learning_rate': [0.01, 0.1, 1],
        'classifier__max_depth': [3, 5]
    }),
    ('MLPClassifier', MLPClassifier(max_iter=1000), {
        'classifier__hidden_layer_sizes': [(50,), (100,)],
        'classifier__activation': ['tanh', 'relu'],
        'classifier__alpha': clf_nn_alpha,
        'classifier__solver' : clf_nn_opt.lower(),
        'classifier__early_stopping' : clf_nn_estop
    })
]

# Define a list of models with parameter grids for regression
regression_models = [
    ('RandomForestRegressor', RandomForestRegressor(), {
        'regressor__n_estimators': reg_rf_n_est,
        'regressor__max_depth': reg_rf_max_depth,
        'regressor__min_samples_leaf': reg_rf_samp_leaf
    }),
    ('GradientBoostingRegressor', GradientBoostingRegressor(), {
        'classifier__n_estimators': clf_gb_n_est,
        'classifier__learning_rate': [0.01, 0.1, 1],
        'classifier__max_depth': clf_gb_minmax_depth
    }),
    ('LinearRegression', LinearRegression(), {
        'classifier__n_jobs' : reg_lreg_njobs
    }),
    ('Ridge', Ridge(), {
        'regressor__alpha': [0.1, 1, 10]
    }),
    ('Lasso', Lasso(), {
        'regressor__alpha': [0.1, 1, 10]
    }),
    ('ElasticNet', ElasticNet(), {
        'regressor__alpha': [0.1, 1, 10],
        'regressor__l1_ratio': [0.2, 0.5, 0.8]
    }),
    ('SVR', SVR(), {
        'regressor__kernel': reg_svr_list,
        'regressor__C': [0.1, 1, 10],
        'regressor__max_iter': reg_svr_max_iter
    }),
    ('SGDRegressor', SGDRegressor(max_iter=1000, tol=1e-3), {
        'regressor__loss': ['squared_loss', 'huber'],
        'regressor__alpha': [0.0001, 0.001, 0.01],
        'regressor__penalty': clf_svc_l_list
    }),
    ('KNeighborsRegressor', KNeighborsRegressor(), {
        'regressor__n_neighbors': [3, 5, 7],
        'regressor__weights': ['uniform', 'distance']
    }),
    ('DecisionTreeRegressor', DecisionTreeRegressor(), {
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [2, 5]
    }),
    ('ExtraTreesRegressor', ExtraTreesRegressor(), {
        'regressor__n_estimators': [50, 100],
        'regressor__max_depth': [None, 10, 20]
    }),
    ('XGBRegressor', XGBRegressor(), {
        'regressor__n_estimators': [50, 100],
        'regressor__learning_rate': [0.01, 0.1, 1],
        'regressor__max_depth': [3, 5]
    }),
    ('MLPRegressor', MLPRegressor(max_iter=1000), {
        'regressor__hidden_layer_sizes': [(50,), (100,)],
        'regressor__activation': ['tanh', 'relu'],
        'regressor__alpha': [0.0001, 0.001]
    })
]


In [91]:
""" Loading the dataset """ 
data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')
# data = pd.read_csv('iris-dataset-dendrite/iris.csv')


# Split the dataset into features and target variable
X = data.drop(columns='species') 
y = data['species']

# Encoding the target
encoder = LabelEncoder()
y = encoder.fit_transform(y)

X = feature_handling(X)

# Setting grid parameters
shuffle = constraint_dict['design_state_data']['hyperparameters']['shuffle_grid']
random_state = constraint_dict['design_state_data']['hyperparameters']['random_state']
splits = constraint_dict['design_state_data']['hyperparameters']['num_of_folds']
njobs = constraint_dict['design_state_data']['hyperparameters']['parallelism']

# Mapping between the input classification algorithm names to the available library function names
class_dict = {'RandomForestClassifier' : 'RandomForestClassifier',
              'GradientBoostingClassifier' : 'GBTClassifier', 
              'LogisticRegression' : 'LogisticRegression',
              'SVC' : 'SVM',
              'SGDClassifier' : 'SGD',
              'KNeighborsClassifier' : 'KNN',
              'DecisionTreeClassifier' : 'DecisionTreeClassifier',
              'ExtraTreesClassifier' : 'extra_random_trees', 
              'XGBClassifier' : 'xg_boost', 
              'MLPClassifier' : 'neural_network'}


# Cross-validation strategies
cv_classification = StratifiedKFold(n_splits=splits, shuffle=shuffle, random_state=random_state)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to perform grid search for classification models
def perform_classification_grid_search(X_train, y_train, models):
    print("*******Classification*********")
    results = []
    for name, model, params in models:
        model_name = re.sub(r"[^\w]", "", name)
#         print(constraint_dict['design_state_data']['algorithms'][class_dict[model_name]]['is_selected'], name)
        if(constraint_dict['design_state_data']['algorithms'][class_dict[model_name]]['is_selected'] == True):
            print(f"Running GridSearchCV for {name}")
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', model)
            ])
            grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_classification, scoring='accuracy', n_jobs=njobs, verbose=2)
            grid_search.fit(X_train, y_train)
            results.append((name, grid_search.best_params_, grid_search.best_score_))
    return results


# Perform grid search for classification models
classification_results = perform_classification_grid_search(X_train, y_train, classification_models)

# Print the results for classification
print("Classification Results:")
for result in classification_results:
    print(f"Model: {result[0]}")
    print(f"Best Parameters: {result[1]}")
    print(f"Best Cross-Validation Accuracy: {result[2]:.2f}")
    print("-" * 30)

# Mapping between the input regression algorithm names to the available library function names
reg_dict = {'RandomForestRegressor':'RandomForestRegressor', 
            'GradientBoostingRegressor' : 'GBTRegressor', 
            'LinearRegression' : 'LinearRegression',
            'Ridge': 'RidgeRegression', 
            'Lasso' : 'LassoRegression', 
            'ElasticNet' : 'ElasticNetRegression', 
            'SVR' : 'SVM', 
            'SGDRegressor' : 'SGD', 
            'KNeighborsRegressor' : 'KNN', 
            'DecisionTreeRegressor' : 'DecisionTreeRegressor', 
            'ExtraTreesRegressor' : 'extra_random_trees',
            'XGBRegressor' : 'xg_boost', 
            'MLPRegressor' : 'neural_network'}

# Function to perform grid search for regression models
def perform_regression_grid_search(X_train, y_train, models):
    print("********Regression*********")
    results = []
    for name, model, params in models:
        model_name = re.sub(r"[^\w]", "", name)
        if(constraint_dict['design_state_data']['algorithms'][reg_dict[model_name]]['is_selected'] == True):
            print(f"Running GridSearchCV for {name}")
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('regressor', model)
            ])
            grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_regression, scoring='neg_mean_squared_error', n_jobs=njobs, verbose=2)
            grid_search.fit(X_train, y_train)
            results.append((name, grid_search.best_params_, grid_search.best_score_))
    return results

# Load regression data from CSV and split
data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')    
X = data.drop(columns = 'species')
X = X.drop(columns= constraint_dict['design_state_data']['target']['target'])
y = data[constraint_dict['design_state_data']['target']['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cv_regression = KFold(n_splits=splits, shuffle=shuffle, random_state=random_state)

# Perform grid search for regression models
regression_results = perform_regression_grid_search(X_train, y_train, regression_models)



# Print the results for regression
print("Regression Results:")
for result in regression_results:
    print(f"Model: {result[0]}")
    print(f"Best Parameters: {result[1]}")
    print(f"Best Cross-Validation MSE: {result[2]:.2f}")
    print("-" * 30)


*******Classification*********
Classification Results:
********Regression*********
Running GridSearchCV for RandomForestRegressor
Fitting 6 folds for each of 12 candidates, totalling 72 fits
Regression Results:
Model: RandomForestRegressor
Best Parameters: {'regressor__max_depth': 25, 'regressor__min_samples_leaf': 5, 'regressor__n_estimators': 15}
Best Cross-Validation MSE: -0.03
------------------------------
